# Préparation des données network

In [1]:
import pandas as pd
import os
from pickleshare import PickleShareDB

## Chargement des données

In [2]:
df_net_1_load = pd.read_csv('../datasets/Network datatset/csv/attack_1.csv')
df_net_2_load = pd.read_csv('../datasets/Network datatset/csv/attack_2.csv')
df_net_3_load = pd.read_csv('../datasets/Network datatset/csv/attack_3.csv')
df_net_4_load = pd.read_csv('../datasets/Network datatset/csv/attack_4.csv')
df_net_norm_load = pd.read_csv('../datasets/Network datatset/csv/normal.csv')

In [3]:
df_net_1 = df_net_1_load.copy()
df_net_2 = df_net_2_load.copy()
df_net_3 = df_net_3_load.copy()
df_net_4 = df_net_4_load.copy()
df_net_norm = df_net_norm_load.copy()

## Enregistrement et ouverture avec PickleShare si besoin

In [4]:
"""
data_dir = '../prep_data' 
os.makedirs(data_dir, exist_ok=True)
db = PickleShareDB(os.path.join(data_dir, 'kity'))

db['net_attack_1'] = df_net_1_load
db['net_attack_2'] = df_net_2_load
db['net_attack_3'] = df_net_3_load
db['net_attack_4'] = df_net_4_load
db['net_norm'] = df_net_norm_load
"""

"\ndata_dir = '../prep_data' \nos.makedirs(data_dir, exist_ok=True)\ndb = PickleShareDB(os.path.join(data_dir, 'kity'))\n\ndb['net_attack_1'] = df_net_1_load\ndb['net_attack_2'] = df_net_2_load\ndb['net_attack_3'] = df_net_3_load\ndb['net_attack_4'] = df_net_4_load\ndb['net_norm'] = df_net_norm_load\n"

In [5]:
"""
df_net_1 = db['net_attack_1']
df_net_2 = db['net_attack_2']
df_net_3 = db['net_attack_3']
df_net_4 = db['net_attack_4']
df_net_norm = db['net_norm']
"""

"\ndf_net_1 = db['net_attack_1']\ndf_net_2 = db['net_attack_2']\ndf_net_3 = db['net_attack_3']\ndf_net_4 = db['net_attack_4']\ndf_net_norm = db['net_norm']\n"

## Nettoyage (1)

In [6]:
df_all = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]
df_nom = ['df_net_1', 'df_net_2', 'df_net_3', 'df_net_4', 'df_net_norm']

### Harmonisation noms colonnes

In [7]:
for df in df_all:
    df.columns = df.columns.str.strip()

### Ordre colonnes

In [8]:
columns_order = ['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto', 'flags', 'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst', 'modbus_response', 'label_n', 'label']
df_all = [df[columns_order] for df in df_all]

## Nettoyage (2)

### Modification des types

In [9]:
for i, df in enumerate(df_all):
    df['label_n'] = df['label_n'].astype('bool')
    df['label'] = df['label'].astype('category')

In [10]:
columns_to_convert = ['sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst']

for i in range(len(df_all)):
    for col in columns_to_convert:
        df_all[i][col] = df_all[i][col].astype('Int64') # ok pour les valeurs manquantes

In [11]:
for i in range(len(df_all)):
    df_all[i]['Time'] = pd.to_datetime(df_all[i]['Time'], format='mixed')

## Nettoyage (3)

### Discrétisation des ports

In [12]:
def get_categorie_ports(port):
    if pd.isna(port):
        return "inconnu" # remplacement des valeurs manquantes
    if port < 1024:
        return "system_ports"
    elif port <= 49151:
        return "user_ports"
    else:
        return "dynamic/private_ports"

In [13]:
for i in range(len(df_all)):
    for col in ['sport', 'dport']:
        df_all[i][col] = df_all[i][col].apply(get_categorie_ports).astype('category')

### Transformation des flags en catégories

In [14]:
for i, df in enumerate(df_all):
    df['flags'] = df['flags'].astype('category')

### One-hot de la colonne flags (faire plus tard)

In [15]:
#for i in range(len(df_all)):
#    df_all[i] = pd.get_dummies(df_all[i], columns=['flags'], prefix='flag')

In [16]:
#df_all[0].head()

Les colonnes créée ne sont pas forcément les même en fonction des datasets, car toutes les valeurs n'ètaient pas présentent dans tous les datasets.  
Nous ajoutons donc des colonnes avec False si elles n'y sont pas.

#### Ajout des colonnes non créées

In [17]:
#all_columns = set()
#for df in df_all:
#    flag_columns = [col for col in df.columns if col.startswith('flag')]
#    all_columns.update(flag_columns)
#all_columns = list(all_columns)

In [18]:
#print(all_columns)

In [19]:
#for i in range(len(df_all)):
#    manquantes = set(all_columns) - set(df_all[i].columns)
#    
#    for col in manquantes:
#        df_all[i][col] = False
#    
#    df_all[i] = df_all[i].reindex(columns=list(df_all[i].columns) + [col for col in all_columns if col not in df_all[i].columns])

In [20]:
# Check si les flags avec des valeurs manquantes ont bien des False dans toutes les colonnes
#columns_to_check = ['flag_0', 'flag_10100', 'flag_10', 'flag_10000', 'flag_11000010', 'flag_11001', 'flag_11000', 'flag_1', 'flag_101001', 'flag_10010', 'flag_100', 'flag_10001']
#
#for i in range(len(df_all)):
#    df = df_all[i]
#    false_count = (~df[columns_to_check].any(axis=1)).sum()
#    print(f"{df_nom[i]} : {false_count} = False")

In [21]:
#df_all[0].head()

## Nettoyage (4)

In [22]:
for df in df_all:
    for col in ['ip_s', 'ip_d', 'modbus_fn', 'modbus_response']:
        df[col] = df[col].fillna("inconnue")
    for col in ['n_pkt_src', 'n_pkt_dst']: # La colonne redevient de type float
        df[col] = df[col].fillna(df[col].median())
    for col in ['flags']:
        df[col] = df[col].cat.add_categories('inconnue')
        df[col] = df[col].fillna('inconnue')

## Enregistrement des données dans les datasets

In [23]:
df_net_1 = df_all[0]
df_net_2 = df_all[1]
df_net_3 = df_all[2]
df_net_4 = df_all[3]
df_net_norm = df_all[4]

In [24]:
df_net_1.head()

,Time,mac_s,mac_d,ip_s,ip_d,sport,dport,proto,flags,size,modbus_fn,n_pkt_src,n_pkt_dst,modbus_response,label_n,label
0,2021-04-09 18:23:28.385003,74:46:a0:bd:a7:1b,0a:fe:ec:47:74:fb,84.3.251.20,84.3.251.102,dynamic/private_ports,system_ports,Modbus,11000,66,Read Coils Request,0,0,inconnue,False,normal
1,2021-04-09 18:23:28.385005,74:46:a0:bd:a7:1b,e6:3f:ac:c9:a8:8c,84.3.251.20,84.3.251.101,dynamic/private_ports,system_ports,Modbus,11000,66,Read Coils Request,1,0,inconnue,False,normal
2,2021-04-09 18:23:28.385006,74:46:a0:bd:a7:1b,fa:00:bc:90:d7:fa,84.3.251.20,84.3.251.103,dynamic/private_ports,system_ports,Modbus,11000,66,Read Coils Request,2,0,inconnue,False,normal
3,2021-04-09 18:23:28.385484,0a:fe:ec:47:74:fb,74:46:a0:bd:a7:1b,84.3.251.102,84.3.251.20,system_ports,dynamic/private_ports,Modbus,11000,64,Read Coils Response,0,0,[0],False,normal
4,2021-04-09 18:23:28.385486,fa:00:bc:90:d7:fa,74:46:a0:bd:a7:1b,84.3.251.103,84.3.251.20,system_ports,dynamic/private_ports,Modbus,11000,64,Read Coils Response,0,1,[0],False,normal


## Enregistrement dans PickleShare

In [25]:
data_dir = '../prep_data' 
os.makedirs(data_dir, exist_ok=True)
db = PickleShareDB(os.path.join(data_dir, 'kity'))

db['net_attack_1_clean'] = df_net_1
db['net_attack_2_clean'] = df_net_2
db['net_attack_3_clean'] = df_net_3
db['net_attack_4_clean'] = df_net_4
db['net_norm_clean'] = df_net_norm